In [1]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./RAG_files/test_demo.csv")
data = loader.load()

In [2]:
import pandas as pd

ex = pd.read_csv('./RAG_files/test_demo.csv')


In [3]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.vectorstores import Chroma

n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
use_mlock = True  # Force system to keep model in RAM.

# Make sure the model path is correct for your system!

from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

embed_model = HuggingFaceEmbeddings(
                model_name=embed_model_id,
                model_kwargs={'device': 'mps'}, 
                encode_kwargs={'device': 'mps'}
)

/Users/zwang/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)


In [5]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed_model)

In [6]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

model_path="/Users/zwang/Desktop/pgpt/privateGPT/models/cache/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/blobs/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9"


# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
    use_mlock = True
)





llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/zwang/Desktop/pgpt/privateGPT/models/cache/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/blobs/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9 (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model

In [7]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": PROMPT},
)


In [9]:
result = qa_chain("how many subbmiter in this file?")['result']

Llama.generate: prefix-match hit


 There are 7 submitters in this file.
Detailed Answer: In the provided context, we can identify seven different submitter names: COLIN HERBERT HYDE, Lulu Monette Johnson, Monette, Denis Hirano, and Colin Herbert Hyde. Therefore, there are a total of 7 submitters in this file.
Note: The detailed answer is based on the provided context, which includes the submitter names listed under "Connected Name." If the context were to change, the answer may also change.


llama_print_timings:        load time =    5075.94 ms
llama_print_timings:      sample time =      30.05 ms /   114 runs   (    0.26 ms per token,  3793.80 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   10584.00 ms /   114 runs   (   92.84 ms per token,    10.77 tokens per second)
llama_print_timings:       total time =   11120.53 ms
